In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sys import path
%matplotlib inline

In [2]:
%cd /content/drive/MyDrive/PHI/ToF_ML/src

/content/drive/MyDrive/PHI/ToF_ML/src


In [17]:
from os import listdir
def get_spectra_tech(dir = '../data/SSL_Csvs/'):
    '''
    Read in better peak info and trandform into lists of channels, intensities, names.
    '''
    fake_names = []
    real_names = []
    techniques = []
    for i, csv in enumerate(listdir(dir)):
        print(csv, i)
        if csv.split('.')[-1] == 'csv':
            data = pd.read_csv(dir + csv, names=list('abcdefgh'))
            techniques.append(data.loc[2]['a'].split(':')[-1])
            fake_names.append(csv[0:-3] + 'cas')
            real_names.append(csv)
    
    return pd.DataFrame(list(zip(fake_names, techniques, real_names)),
    columns=['file_name', 'technique', 'correct_file_name'])

In [24]:
dir = '../data/'
folders = ['SSL_Csvs/', '554_CalibratedSpectraFromGreg/']

In [26]:
df2 = get_spectra_tech(dir + folders[1])

091405-70.tdc 0
091405-69.tdc 1
MS2-Display Bi3+ Bunched 200umFOV 551-MS2-50%-007-Pos.csv 2
Mounting Tape Pos-031.tdc 3
P1_P_-03.csv 4
P2_N_-01.csv 5
091405-79.tdc 6
N1B 200umFOV top C60 NEG.tdc 7
N1B 250umFOV trench POS.csv 8
N1C 200umFOV top C60 NEG.tdc 9
091405-78.tdc 10
N1B 100umFOV trench NEG.csv 11
N1B 200umFOV top C60 NEG.csv 12
MS2-Display Bi3+ Bunched 400umFOV 552-MS2-75%-026-Pos~pb1.tdc 13
N1C 100umFOV trench NEG.csv 14
MS2-Display Bi3+ Bunched 400umFOV 552-MS2-75%-035-Pos.tdc 15
MS2-Display Bi3+ Bunched 400umFOV 552-MS2-75%-026-Pos~pb1.csv 16
N1C 250umFOV trench POS.csv 17
N1C 200umFOV top C60 NEG.csv 18
091405-77.tdc 19
MS2-Display Bi3+ Bunched 400umFOV 552-MS2-75%-026-Pos.csv 20
MS2-Display Bi3+ Bunched 400umFOV 551-MS2-75%-027-Pos~pb1.csv 21
MS2-Display Bi3+ Bunched 400umFOV 522-MS2-75%-028-Pos.tdc 22
MS2-Display Bi3+ Bunched 400umFOV 552-MS2-75%-035-Pos.csv 23
MS2-Display Bi3+ Bunched 400umFOV 552-MS2-75%-026-Pos.tdc 24
MS2-Display Bi3+ Bunched 400umFOV 522-MS2-75%-028-P

In [20]:
df.head()

,file_name,technique,correct_file_name
0,C0059-V1.cas,-TofSIMS,C0059-V1.csv
1,C0060-V1.cas,-TofSIMS,C0060-V1.csv
2,C0061-U1.cas,+TofSIMS,C0061-U1.csv
3,C0061-V1.cas,-TofSIMS,C0061-V1.csv
4,C0062-U1.cas,+TofSIMS,C0062-U1.csv


In [27]:
df2.head()

,file_name,technique,correct_file_name
0,MS2-Display Bi3+ Bunched 200umFOV 551-MS2-50%-...,+TofSIMS,MS2-Display Bi3+ Bunched 200umFOV 551-MS2-50%-...
1,P1_P_-03.cas,+TofSIMS,P1_P_-03.csv
2,P2_N_-01.cas,-TofSIMS,P2_N_-01.csv
3,N1B 250umFOV trench POS.cas,+TofSIMS,N1B 250umFOV trench POS.csv
4,N1B 100umFOV trench NEG.cas,-TofSIMS,N1B 100umFOV trench NEG.csv


In [28]:
len(df2)

554

In [29]:
len(df)

1912

In [30]:
df2.to_csv('../data/552gregspectratechs.csv', index=False)

In [31]:
df.to_csv('../data/newspectratechs.csv', index=False)

In [33]:
%pycat process_cas.py

In [36]:
df3 = pd.read_csv('../data/processed_cas.csv')

In [47]:
df3 = df3[['file_name', 'Technique']].copy()
df3.columns = ['file_name', 'technique']

In [49]:
tech_df = pd.concat([df, df2, df3])

In [44]:
len(df3)

906

In [62]:
def get_tech(tech):
    if tech.strip()[0] == '+':
        return 1
    else:
        return 0

In [55]:
tech_df.reset_index(inplace=True, drop=True)

In [63]:
tech_df['pos'] = tech_df['technique'].apply(get_tech)

In [65]:
tech_df.tail()

,file_name,technique,correct_file_name,pos
3367,CHB149NF.cas,-ToFSIMS\n,NaN,0
3368,0617214.cas,+ToFSIMS\n,NaN,1
3369,CHC37PC.cas,+ToFSIMS\n,NaN,1
3370,ZD15_02.cas,-ToFSIMS\n,NaN,0
3371,CHC14NB.cas,-ToFSIMS\n,NaN,0


In [61]:
tech_df.loc[0]['technique'].strip()[0]

'-'

In [78]:
fd = pd.read_csv('../data/full_corrected_data.csv')

In [79]:
from ast import literal_eval
fd['channels'] = fd['channels'].apply(literal_eval)
fd['intensities'] = fd['intensities'].apply(literal_eval)
fd['masses'] = fd['masses'].apply(literal_eval)

In [80]:
fd.columns

Index(['file_name', 'MassOverTime', 'MassOffset', 'StartFlightTime',
       'SpecBinSize', 'channels', 'intensities', 'masses',
       'adjusted_original_proportion_identified', 'avg_dist_frags_low',
       'avg_dist_frags_high', 'adjusted_proportion_identified',
       'proportion_identified', 'diff', 'prop_diff_in_low', 'calibration'],
      dtype='object')

In [81]:
new = pd.DataFrame(columns=['file_name', 'MassOverTime', 'MassOffset', 'StartFlightTime',
       'SpecBinSize','channels', 'intensities', 'masses',
       'adjusted_original_proportion_identified', 'avg_dist_frags_low',
       'avg_dist_frags_high', 'adjusted_proportion_identified',
       'proportion_identified', 'diff', 'prop_diff_in_low', 'calibration', 'technique',  ])

In [82]:
i = 0
for row in tech_df.itertuples():
    ind = fd[fd['file_name'] == row.file_name].index
    if len(ind) == 1:
        data = fd.loc[ind[0]].copy()
        if row.correct_file_name != 'NaN':
            data['file_name'] = row.correct_file_name
        data['technique'] = row.pos
        new.loc[i] = data
        i += 1

In [74]:
fd[fd['file_name'] == tech_df.loc[0].file_name].index[0]

1437

In [83]:
new.head()

,file_name,MassOverTime,MassOffset,StartFlightTime,SpecBinSize,channels,intensities,masses,adjusted_original_proportion_identified,avg_dist_frags_low,avg_dist_frags_high,adjusted_proportion_identified,proportion_identified,diff,prop_diff_in_low,calibration,technique
0,C0059-V1.csv,1.690970,0.708925,0.0,0.128,"[12729.6134, 13388.07292, 14021.38724, 14625.2...","[10106.0, 32018.0, 4303.0, 816.0, 44497.0, 346...","[12.000516134304917, 13.00825496659651, 14.015...",0.307692,0.000930,0.001400,0.538462,0.515789,0.000470,0.504991,0,0
1,C0060-V1.csv,1.686650,0.706374,0.0,0.128,"[12773.45818, 13433.49441, 13495.05187, 14068....","[37242.0, 67973.0, 112.0, 5361.0, 4489.0, 2193...","[11.999648578820876, 13.007180420340381, 13.10...",0.409091,0.000994,0.001364,0.562937,0.542088,0.000370,0.372432,1,0
2,C0061-U1.csv,1.684820,0.708105,0.0,0.128,"[12779.34033, 13440.22492, 14076.03163, 14689....","[483.0, 782.0, 1374.0, 2985.0, 152.0, 302.0, 3...","[11.999700337252365, 13.007438841283296, 14.01...",0.428571,0.001181,0.001583,0.575188,0.558394,0.000402,0.340298,1,1
3,C0061-V1.csv,1.690380,0.708417,0.0,0.128,"[1381.98178, 1400.517, 1412.53457, 1440.11759,...","[2675.0, 1381.0, 674.0, 196.0, 93.0, 50539.0, ...","[1.014923540233473, 1.0230201316204572, 1.0282...",0.305155,0.001156,0.001566,0.381443,0.302288,0.000410,0.354811,0,0
4,C0062-U1.csv,1.683051,0.707803,0.0,0.128,"[12795.06881, 13456.57005, 14092.96684, 14707....","[565.0, 835.0, 1243.0, 2532.0, 99.0, 220.0, 14...","[12.0010393218355, 13.008711984543739, 14.0164...",0.427509,0.001284,0.001697,0.609665,0.605166,0.000412,0.321084,1,1


In [92]:
len(new)

3348

In [85]:
len(fd)

3348

In [86]:
new.to_csv('../data/full_corrected_dataset_with_techs.csv', index=False)

In [113]:
from data_transformation import get_frags
frags = get_frags()['FragmentMass']
positives = [0 for x in range(len(frags))]
negatives = [0 for x in range(len(frags))]
tech_list = [negatives, positives]

In [114]:
from data_transformation import get_frags_dists
for row in new.itertuples():
    masses, fragments, distances = get_frags_dists(row.masses, frags, threshes=[0.003])
    for frag in fragments:
        ind = frags[frags == frag].index
        if len(ind) == 1:
            tech_list[row.technique][ind[0]] += 1

In [124]:
negatives = np.array(negatives)
positives = np.array(positives)

In [123]:
fragment_df = get_frags()

In [ ]:
fragment_df['occurrences'] = positives + negatives

In [126]:
fragment_df['negative_occurence_rate'] = negatives / (negatives + positives + 1)

In [127]:
fragment_df['positive_occurence_rate'] = positives / (negatives + positives + 1)

In [132]:
fragment_df.head(20)

,FragmentMass,FragmentLabel,Isotopes,Formula,FragmentID,negative_occurence_rate,positive_occurence_rate,occurrences
0,1.007820,H,1H1,H,1,0.400000,0.599048,1049
1,2.014100,H,2H1,2H,2,0.389216,0.609804,1019
2,3.016029,He,3He1,3He,3,0.166667,0.791667,23
3,4.002603,He,4He1,He,4,0.000000,0.000000,0
4,6.015120,Li,6Li1,6Li,5,0.052632,0.934211,75
5,7.016000,Li,7Li1,Li,6,0.071090,0.924171,210
6,9.012180,Be,9Be1,Be,7,0.142857,0.714286,6
7,10.012940,B,10B1,10B,8,0.120370,0.870370,107
8,11.009300,B,11B1,B,9,0.128571,0.866667,209
9,12.000000,C,12C1,C,10,0.430563,0.568901,1864


In [134]:
fragment_df.to_csv('../data/fragment_data_with_pos_neg_occurrences.csv', index=False)

Overwriting data_transformation.py
